In [3]:
mkdir -p semeval26-05-scripts/input/ref


In [4]:
%%writefile semeval26-05-scripts/scoring.py
import json
import math
from scipy.stats import spearmanr

def load_jsonl(path):
    data = {}
    with open(path, "r") as f:
        for line in f:
            obj = json.loads(line)
            data[obj["id"]] = obj
    return data

def main(ref_path, pred_path, out_path):
    ref = load_jsonl(ref_path)
    pred = load_jsonl(pred_path)

    gold_scores = []
    pred_scores = []
    within_std = []

    for _id, gold in ref.items():
        if _id not in pred:
            continue

        g = gold["mean"]
        sd = gold.get("std", 1)
        p = pred[_id]["prediction"]

        gold_scores.append(g)
        pred_scores.append(p)

        if abs(p - g) <= sd:
            within_std.append(1)
        else:
            within_std.append(0)

    # spearman
    spearman = spearmanr(gold_scores, pred_scores).correlation
    if math.isnan(spearman):
        spearman = 0

    acc = sum(within_std) / len(within_std)

    results = {
        "spearman": float(spearman),
        "within_std_accuracy": float(acc)
    }

    with open(out_path, "w") as f:
        json.dump(results, f, indent=2)

    print(results)

if __name__ == "__main__":
    import sys
    main(sys.argv[1], sys.argv[2], sys.argv[3])


Writing semeval26-05-scripts/scoring.py


In [5]:
%%writefile semeval26-05-scripts/format_check.py
import json, sys

path = sys.argv[1]

with open(path) as f:
    for i, line in enumerate(f):
        try:
            obj = json.loads(line)
        except:
            print("Line", i, "is not valid JSON")
            exit(1)

        if "id" not in obj:
            print("Missing 'id' at line", i)
            exit(1)
        if "prediction" not in obj:
            print("Missing 'prediction' at line", i)
            exit(1)

print("Format OK")


Writing semeval26-05-scripts/format_check.py


In [6]:
%%writefile generate_solution_from_dev.py
import json

dev = json.load(open("dev.json"))
out = open("semeval26-05-scripts/input/ref/solution.jsonl", "w")

for id_, row in dev.items():
    obj = {
        "id": id_,
        "mean": float(row["mean_score"]),   # adjust field
        "std": float(row["std_score"]),     # adjust field if exists
    }
    out.write(json.dumps(obj) + "\n")

out.close()
print("Created solution.jsonl")


Writing generate_solution_from_dev.py


In [9]:
!ls -R /kaggle/working


/kaggle/working:
generate_solution_from_dev.py  semeval26-05-scripts

/kaggle/working/semeval26-05-scripts:
format_check.py  input	scoring.py

/kaggle/working/semeval26-05-scripts/input:
ref

/kaggle/working/semeval26-05-scripts/input/ref:


In [12]:
!ls -R /kaggle/input/semeval26-task5-dataa


/kaggle/input/semeval26-task5-dataa:
dev.json  sample_data.json  train.json


In [15]:
!mkdir -p /kaggle/working/src


In [16]:
%%writefile /kaggle/working/src/dataset.py
import json
from datasets import Dataset

LABEL_FIELD = "average"
STD_FIELD = "stdev"

TEXT_FIELDS = [
    "precontext",
    "sentence",
    "ending",
    "example_sentence",
    "judged_meaning",
    "homonym"
]

def _normalize(path):
    data = json.load(open(path, "r"))
    samples = []

    if isinstance(data, dict):
        for k, ex in data.items():
            ex = dict(ex)
            ex["id"] = str(k)
            samples.append(ex)
    else:
        for i, ex in enumerate(data):
            ex = dict(ex)
            ex.setdefault("id", str(i))
            samples.append(ex)

    return samples


def _build_text(ex):
    parts = []
    for field in TEXT_FIELDS:
        if field in ex and isinstance(ex[field], str):
            parts.append(ex[field])
    return " ".join(parts)


def load_dataset(path):
    raw = _normalize(path)
    processed = []

    for ex in raw:
        row = {
            "id": ex["id"],
            "text": _build_text(ex),
        }
        if LABEL_FIELD in ex:
            row[LABEL_FIELD] = float(ex[LABEL_FIELD])
        if STD_FIELD in ex:
            row[STD_FIELD] = float(ex[STD_FIELD])
        processed.append(row)

    return Dataset.from_list(processed)


Writing /kaggle/working/src/dataset.py


In [17]:
%%writefile /kaggle/working/generate_solution_from_dev.py
import json

dev = json.load(open("/kaggle/input/semeval26-task5-dataa/dev.json"))
out = open("/kaggle/working/semeval26-05-scripts/input/ref/solution.jsonl", "w")

for _id, row in dev.items():
    obj = {
        "id": str(_id),
        "mean": float(row["average"]),
        "std": float(row["stdev"])
    }
    out.write(json.dumps(obj) + "\n")

out.close()
print("Created solution.jsonl")


Overwriting /kaggle/working/generate_solution_from_dev.py


In [18]:
!python /kaggle/working/generate_solution_from_dev.py


Created solution.jsonl


In [25]:
!ls /kaggle/input


roberta-base-model  semeval26-task5-dataa


In [26]:
!ls -R /kaggle/input/roberta-base-model


/kaggle/input/roberta-base-model:
other

/kaggle/input/roberta-base-model/other:
default

/kaggle/input/roberta-base-model/other/default:
1

/kaggle/input/roberta-base-model/other/default/1:
roberta-base

/kaggle/input/roberta-base-model/other/default/1/roberta-base:
config.json  model.safetensors	      tokenizer_config.json  vocab.json
merges.txt   special_tokens_map.json  tokenizer.json


In [33]:
%%writefile /kaggle/working/src/train_model.py
import os
import numpy as np
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from scipy.stats import spearmanr
from datasets import DatasetDict
from dataset import load_dataset, LABEL_FIELD

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Your offline model path (correct)
MODEL_NAME = "/kaggle/input/roberta-base-model/other/default/1/roberta-base"

OUT_DIR = "/kaggle/working/models/semeval_roberta"
MAX_LEN = 256


def tokenize(batch, tokenizer):
    enc = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )
    enc["labels"] = batch[LABEL_FIELD]
    return enc


def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds[:, 0] if preds.ndim > 1 else preds
    preds = np.clip(np.rint(preds), 1, 5)
    s = spearmanr(preds, labels).correlation
    return {"spearman": float(0 if np.isnan(s) else s)}


def main():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    train = load_dataset("/kaggle/input/semeval26-task5-dataa/train.json")
    dev = load_dataset("/kaggle/input/semeval26-task5-dataa/dev.json")

    tokenized = DatasetDict({
        "train": train.map(lambda b: tokenize(b, tokenizer), batched=True),
        "dev": dev.map(lambda b: tokenize(b, tokenizer), batched=True)
    })

    # Load model but ignore mismatched classifier weights
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=1,
        problem_type="regression",
        ignore_mismatched_sizes=True
    )

    # Kaggle-safe TrainingArguments (older transformers compatible)
    args = TrainingArguments(
        output_dir=OUT_DIR,
        do_train=True,
        do_eval=True,
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_steps=100,
        save_steps=500,
        report_to=[]  # disable WandB
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["dev"],
        compute_metrics=compute_metrics,
    )

    trainer.train()
    trainer.save_model(OUT_DIR)
    tokenizer.save_pretrained(OUT_DIR)


if __name__ == "__main__":
    main()


Overwriting /kaggle/working/src/train_model.py


In [35]:
!python /kaggle/working/src/train_model.py


2025-11-25 00:52:07.827095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764031927.844600     189 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764031927.849577     189 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
Map: 100%|███████████████████████████| 588/588 [00:00<00:00, 3646.71 examples

In [37]:
%%writefile /kaggle/working/src/generate_solution_from_dev.py
import json
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from dataset import load_dataset, LABEL_FIELD
import torch

MODEL_PATH = "/kaggle/working/models/semeval_roberta"   # path after training
DEV_PATH = "/kaggle/input/semeval26-task5-dataa/dev.json"
OUT_FILE = "/kaggle/working/predictions_dev.jsonl"

def main():
    print("Loading model...")
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_PATH, num_labels=1, problem_type="regression"
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

    print("Loading dev set...")
    dev = load_dataset(DEV_PATH)

    preds = []

    print("Generating predictions...")
    for ex in dev:
        inputs = tokenizer(
            ex["text"],
            truncation=True,
            padding=True,
            max_length=256,
            return_tensors="pt"
        )

        with torch.no_grad():
            output = model(**inputs)
            score = output.logits.squeeze().item()

        # Clip + round to valid SemEval range 1–5
        score = float(np.clip(round(score), 1, 5))

        preds.append({"id": ex["id"], "pred": score})

    print(f"Writing to {OUT_FILE} ...")
    with open(OUT_FILE, "w") as f:
        for p in preds:
            f.write(json.dumps(p) + "\n")

    print("Done! File saved:", OUT_FILE)

if __name__ == "__main__":
    main()


Writing /kaggle/working/src/generate_solution_from_dev.py


In [38]:
!python /kaggle/working/src/generate_solution_from_dev.py


Loading model...
2025-11-25 03:04:26.748261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764039866.805306     211 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764039866.823948     211 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
Loading dev set...
Generating predictions...
Writing to /kag

In [41]:
import json
from pprint import pprint

with open("/kaggle/input/semeval26-task5-dataa/dev.json") as f:
    dev = json.load(f)

first = list(dev.items())[0]
pprint(first)


('0',
 {'average': 3.6,
  'choices': [4, 5, 3, 1, 5],
  'ending': 'They began to run along the abandoned railway line, hopping from '
            'wooden sleeper to sleeper to avoid twisting an ankle.',
  'example_sentence': 'The train glided smoothly along the track.',
  'homonym': 'track',
  'judged_meaning': 'a pair of parallel rails providing a runway for wheels',
  'nonsensical': [False, False, False, False, False],
  'precontext': 'The detectives arrived at the abandoned train station. They '
                'were looking for signs of the missing artifact. A faint trail '
                'caught their attention.',
  'sample_id': '2371',
  'sentence': 'They followed the track.',
  'stdev': 1.6733200530681511})


In [42]:
!head -n 20 /kaggle/working/predictions_dev.jsonl


{"id": "0", "pred": 4.0}
{"id": "1", "pred": 3.0}
{"id": "2", "pred": 3.0}
{"id": "3", "pred": 4.0}
{"id": "4", "pred": 3.0}
{"id": "5", "pred": 3.0}
{"id": "6", "pred": 5.0}
{"id": "7", "pred": 3.0}
{"id": "8", "pred": 5.0}
{"id": "9", "pred": 5.0}
{"id": "10", "pred": 4.0}
{"id": "11", "pred": 4.0}
{"id": "12", "pred": 4.0}
{"id": "13", "pred": 4.0}
{"id": "14", "pred": 4.0}
{"id": "15", "pred": 4.0}
{"id": "16", "pred": 4.0}
{"id": "17", "pred": 4.0}
{"id": "18", "pred": 5.0}
{"id": "19", "pred": 4.0}


In [43]:
%%writefile /kaggle/working/src/fix_predictions.py
import json

input_path = "/kaggle/working/predictions_dev.jsonl"
output_path = "/kaggle/working/predictions_dev_fixed.jsonl"

with open(input_path, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        obj = json.loads(line)
        # extract score, rename field
        score = float(obj["pred"])
        fout.write(json.dumps({"prediction": score}) + "\n")

print("Fixed file saved to:", output_path)


Writing /kaggle/working/src/fix_predictions.py


In [44]:
!python /kaggle/working/src/fix_predictions.py


Fixed file saved to: /kaggle/working/predictions_dev_fixed.jsonl


In [45]:
!head -n 20 /kaggle/working/predictions_dev_fixed.jsonl


{"prediction": 4.0}
{"prediction": 3.0}
{"prediction": 3.0}
{"prediction": 4.0}
{"prediction": 3.0}
{"prediction": 3.0}
{"prediction": 5.0}
{"prediction": 3.0}
{"prediction": 5.0}
{"prediction": 5.0}
{"prediction": 4.0}
{"prediction": 4.0}
{"prediction": 4.0}
{"prediction": 4.0}
{"prediction": 4.0}
{"prediction": 4.0}
{"prediction": 4.0}
{"prediction": 4.0}
{"prediction": 5.0}
{"prediction": 4.0}


In [51]:
!ls -l /kaggle/working/semeval26-05-scripts/input/ref/solution.jsonl


-rw-r--r-- 1 root root 30597 Nov 25 00:31 /kaggle/working/semeval26-05-scripts/input/ref/solution.jsonl


In [52]:
!head -n 20 /kaggle/working/semeval26-05-scripts/input/ref/solution.jsonl


{"id": "0", "mean": 3.6, "std": 1.6733200530681511}
{"id": "1", "mean": 3.6, "std": 0.5477225575051661}
{"id": "2", "mean": 3.8, "std": 1.3038404810405297}
{"id": "3", "mean": 4.2, "std": 0.8366600265340756}
{"id": "4", "mean": 3.0, "std": 1.8708286933869707}
{"id": "5", "mean": 3.0, "std": 1.224744871391589}
{"id": "6", "mean": 4.6, "std": 0.5477225575051661}
{"id": "7", "mean": 1.3333333333333333, "std": 0.5163977794943223}
{"id": "8", "mean": 2.2, "std": 1.3038404810405297}
{"id": "9", "mean": 3.8, "std": 1.0954451150103321}
{"id": "10", "mean": 3.2, "std": 1.4832396974191326}
{"id": "11", "mean": 2.2, "std": 1.6431676725154984}
{"id": "12", "mean": 4.0, "std": 1.0}
{"id": "13", "mean": 3.0, "std": 1.5811388300841898}
{"id": "14", "mean": 2.2, "std": 0.8366600265340756}
{"id": "15", "mean": 2.4, "std": 1.140175425099138}
{"id": "16", "mean": 3.4, "std": 1.816590212458495}
{"id": "17", "mean": 2.0, "std": 0.8944271909999159}
{"id": "18", "mean": 4.6, "std": 0.8944271909999159}
{"id":

In [55]:
!sed -n '1p' /kaggle/working/predictions_dev_fixed.jsonl | od -c


0000000   {   "   p   r   e   d   i   c   t   i   o   n   "   :       4
0000020   .   0   }  \n
0000024


In [57]:
!ls -R /kaggle/working/semeval26-05-scripts


/kaggle/working/semeval26-05-scripts:
format_check.py  input	scoring.py

/kaggle/working/semeval26-05-scripts/input:
ref

/kaggle/working/semeval26-05-scripts/input/ref:
solution.jsonl


In [58]:
!sed -n '1,50p' /kaggle/working/semeval26-05-scripts/input/ref/solution.jsonl


{"id": "0", "mean": 3.6, "std": 1.6733200530681511}
{"id": "1", "mean": 3.6, "std": 0.5477225575051661}
{"id": "2", "mean": 3.8, "std": 1.3038404810405297}
{"id": "3", "mean": 4.2, "std": 0.8366600265340756}
{"id": "4", "mean": 3.0, "std": 1.8708286933869707}
{"id": "5", "mean": 3.0, "std": 1.224744871391589}
{"id": "6", "mean": 4.6, "std": 0.5477225575051661}
{"id": "7", "mean": 1.3333333333333333, "std": 0.5163977794943223}
{"id": "8", "mean": 2.2, "std": 1.3038404810405297}
{"id": "9", "mean": 3.8, "std": 1.0954451150103321}
{"id": "10", "mean": 3.2, "std": 1.4832396974191326}
{"id": "11", "mean": 2.2, "std": 1.6431676725154984}
{"id": "12", "mean": 4.0, "std": 1.0}
{"id": "13", "mean": 3.0, "std": 1.5811388300841898}
{"id": "14", "mean": 2.2, "std": 0.8366600265340756}
{"id": "15", "mean": 2.4, "std": 1.140175425099138}
{"id": "16", "mean": 3.4, "std": 1.816590212458495}
{"id": "17", "mean": 2.0, "std": 0.8944271909999159}
{"id": "18", "mean": 4.6, "std": 0.8944271909999159}
{"id":

In [59]:
%%writefile /kaggle/working/fix_reference.py
import json
import numpy as np

input_path = "/mnt/data/solution.jsonl"
output_path = "/kaggle/working/solution.jsonl"

with open(input_path) as fin, open(output_path, "w") as fout:
    for line in fin:
        obj = json.loads(line)
        labels = obj["label"]
        mean = float(np.mean(labels))
        std = float(np.std(labels, ddof=0))
        fout.write(json.dumps({
            "id": obj["id"],
            "mean": mean,
            "std": std
        }) + "\n")

print("Corrected reference file written to:", output_path)


Writing /kaggle/working/fix_reference.py


In [63]:
!sed -n '1,30p' /kaggle/input/solution-fixed/solution_fixed.jsonl


{"id": "0", "mean": 3.6, "std": 1.4966629547095764}
{"id": "1", "mean": 3.6, "std": 0.4898979485566356}
{"id": "2", "mean": 3.8, "std": 1.16619037896906}
{"id": "3", "mean": 4.2, "std": 0.7483314773547882}
{"id": "4", "mean": 3.0, "std": 1.6733200530681511}
{"id": "5", "mean": 3.0, "std": 1.0954451150103321}
{"id": "6", "mean": 4.6, "std": 0.48989794855663565}
{"id": "7", "mean": 1.3333333333333333, "std": 0.4714045207910317}
{"id": "8", "mean": 2.2, "std": 1.16619037896906}
{"id": "9", "mean": 3.8, "std": 0.9797958971132712}
{"id": "10", "mean": 3.2, "std": 1.32664991614216}
{"id": "11", "mean": 2.2, "std": 1.469693845669907}
{"id": "12", "mean": 4.0, "std": 0.8944271909999159}
{"id": "13", "mean": 3.0, "std": 1.4142135623730951}
{"id": "14", "mean": 2.2, "std": 0.7483314773547882}
{"id": "15", "mean": 2.4, "std": 1.019803902718557}
{"id": "16", "mean": 3.4, "std": 1.624807680927192}
{"id": "17", "mean": 2.0, "std": 0.816496580927726}
{"id": "18", "mean": 4.6, "std": 0.799999999999999

In [64]:
!ls /kaggle/input


roberta-base-model  semeval26-task5-dataa  solution-fixed


In [65]:
!ls -R /kaggle/input/solution-fixed


/kaggle/input/solution-fixed:
solution_fixed.jsonl


In [66]:
!sed -n '1,20p' /kaggle/input/solution-fixed/solution_fixed.jsonl


{"id": "0", "mean": 3.6, "std": 1.4966629547095764}
{"id": "1", "mean": 3.6, "std": 0.4898979485566356}
{"id": "2", "mean": 3.8, "std": 1.16619037896906}
{"id": "3", "mean": 4.2, "std": 0.7483314773547882}
{"id": "4", "mean": 3.0, "std": 1.6733200530681511}
{"id": "5", "mean": 3.0, "std": 1.0954451150103321}
{"id": "6", "mean": 4.6, "std": 0.48989794855663565}
{"id": "7", "mean": 1.3333333333333333, "std": 0.4714045207910317}
{"id": "8", "mean": 2.2, "std": 1.16619037896906}
{"id": "9", "mean": 3.8, "std": 0.9797958971132712}
{"id": "10", "mean": 3.2, "std": 1.32664991614216}
{"id": "11", "mean": 2.2, "std": 1.469693845669907}
{"id": "12", "mean": 4.0, "std": 0.8944271909999159}
{"id": "13", "mean": 3.0, "std": 1.4142135623730951}
{"id": "14", "mean": 2.2, "std": 0.7483314773547882}
{"id": "15", "mean": 2.4, "std": 1.019803902718557}
{"id": "16", "mean": 3.4, "std": 1.624807680927192}
{"id": "17", "mean": 2.0, "std": 0.816496580927726}
{"id": "18", "mean": 4.6, "std": 0.799999999999999

In [69]:
!head /kaggle/working/predictions_dev_fixed.jsonl


{"prediction": 4.0}
{"prediction": 3.0}
{"prediction": 3.0}
{"prediction": 4.0}
{"prediction": 3.0}
{"prediction": 3.0}
{"prediction": 5.0}
{"prediction": 3.0}
{"prediction": 5.0}
{"prediction": 5.0}


In [71]:
import json

inp = "/kaggle/working/predictions_dev_fixed.jsonl"
out = "/kaggle/working/predictions_dev_fixed_with_id.jsonl"

with open(inp) as fin, open(out, "w") as fout:
    for i, line in enumerate(fin):
        obj = json.loads(line)
        obj["id"] = str(i)     # assign id matching reference
        fout.write(json.dumps(obj) + "\n")


In [73]:
!head /kaggle/working/predictions_dev_fixed_with_id.jsonl


{"prediction": 4.0, "id": "0"}
{"prediction": 3.0, "id": "1"}
{"prediction": 3.0, "id": "2"}
{"prediction": 4.0, "id": "3"}
{"prediction": 3.0, "id": "4"}
{"prediction": 3.0, "id": "5"}
{"prediction": 5.0, "id": "6"}
{"prediction": 3.0, "id": "7"}
{"prediction": 5.0, "id": "8"}
{"prediction": 5.0, "id": "9"}


In [77]:
!wc -l /kaggle/working/predictions_dev_fixed.jsonl
!wc -l /kaggle/input/solution-fixed/solution_fixed.jsonl


588 /kaggle/working/predictions_dev_fixed.jsonl
588 /kaggle/input/solution-fixed/solution_fixed.jsonl


In [78]:
!head /kaggle/working/predictions_dev_fixed_with_id.jsonl


{"prediction": 4.0, "id": "0"}
{"prediction": 3.0, "id": "1"}
{"prediction": 3.0, "id": "2"}
{"prediction": 4.0, "id": "3"}
{"prediction": 3.0, "id": "4"}
{"prediction": 3.0, "id": "5"}
{"prediction": 5.0, "id": "6"}
{"prediction": 3.0, "id": "7"}
{"prediction": 5.0, "id": "8"}
{"prediction": 5.0, "id": "9"}


In [79]:
!tail /kaggle/working/predictions_dev_fixed_with_id.jsonl


{"prediction": 3.0, "id": "578"}
{"prediction": 3.0, "id": "579"}
{"prediction": 3.0, "id": "580"}
{"prediction": 3.0, "id": "581"}
{"prediction": 3.0, "id": "582"}
{"prediction": 3.0, "id": "583"}
{"prediction": 3.0, "id": "584"}
{"prediction": 3.0, "id": "585"}
{"prediction": 3.0, "id": "586"}
{"prediction": 3.0, "id": "587"}


In [83]:
import json

bad = []
with open("/kaggle/input/solution-fixed/solution_fixed.jsonl") as f:
    for i, line in enumerate(f):
        try:
            obj = json.loads(line)
            if "mean" not in obj:
                bad.append((i, obj))
        except:
            bad.append((i, "INVALID JSON"))

bad[:20], len(bad)


([], 0)

In [84]:
!python /kaggle/working/semeval26-05-scripts/scoring.py \
    /kaggle/input/solution-fixed/solution_fixed.jsonl \
    /kaggle/working/predictions_dev_fixed_with_id.jsonl \
    /kaggle/working/scores.json


{'spearman': 0.3190936738480133, 'within_std_accuracy': 0.4744897959183674}
